In [ ]:
import numpy as np
import os
import time
import mediapipe as mp
import cv2
import matplotlib.pyplot as plt

In [ ]:
mpipe_holistic = mp.solutions.holistic
mpipe_drawing = mp.solutions.drawing_utils

In [ ]:
def mpipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    result = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    return result, image

In [ ]:
def draw_landmarks(image, result):
    # For face landmarks
    mpipe_drawing.draw_landmarks(image, result.face_landmarks, mpipe_holistic.FACEMESH_CONTOURS,
                              mpipe_drawing.DrawingSpec(color=(0, 0, 255), thickness=1, circle_radius=1),
                              mpipe_drawing.DrawingSpec(color=(0, 255, 0), thickness=1, circle_radius=1),
                             )
    
    # For pose landmarks
    mpipe_drawing.draw_landmarks(image, result.pose_landmarks, mpipe_holistic.POSE_CONNECTIONS,
                              mpipe_drawing.DrawingSpec(color=(0, 0, 255), thickness=1, circle_radius=1),
                              mpipe_drawing.DrawingSpec(color=(0, 255, 0), thickness=1, circle_radius=1),
                             )
    
    # For left hand landmarks
    mpipe_drawing.draw_landmarks(image, result.left_hand_landmarks, mpipe_holistic.HAND_CONNECTIONS,
                              mpipe_drawing.DrawingSpec(color=(0, 0, 255), thickness=1, circle_radius=4),
                              mpipe_drawing.DrawingSpec(color=(255, 255, 0), thickness=1, circle_radius=2),
                             )
    
    # For right hand landmarks
    mpipe_drawing.draw_landmarks(image, result.right_hand_landmarks, mpipe_holistic.HAND_CONNECTIONS,
                              mpipe_drawing.DrawingSpec(color=(0, 0, 255), thickness=1, circle_radius=4),
                              mpipe_drawing.DrawingSpec(color=(0, 255, 255), thickness=1, circle_radius=2),
                             )    

In [ ]:
# capture = cv2.VideoCapture(0)
# with mpipe_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
#     while(capture.isOpened()):
#         ret, frame = capture.read()
        
#         result, img = mpipe_detection(frame, holistic)
#         draw_landmarks(img, result)
        
#         cv2.imshow("Video", img)

#         if cv2.waitKey(10) & 0xFF == ord('q'):
#             break

#     capture.release()
#     cv2.destroyAllWindows()

In [ ]:
def to_array_keypoints(res_landmarks):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in res_landmarks.pose_landmarks.landmark]).flatten() if res_landmarks.pose_landmarks else np.zeros(33 * 4)
    left_hand = np.array([[res.x, res.y, res.z] for res in res_landmarks.left_hand_landmarks.landmark]).flatten() if res_landmarks.left_hand_landmarks else np.zeros(21 * 3)
    right_hand = np.array([[res.x, res.y, res.z] for res in res_landmarks.right_hand_landmarks.landmark]).flatten() if res_landmarks.right_hand_landmarks else np.zeros(21 * 3)
    face = np.array([[res.x, res.y, res.z] for res in res_landmarks.face_landmarks.landmark]).flatten() if res_landmarks.face_landmarks else np.zeros(468 * 3)
    
    return np.concatenate([face, pose, left_hand, right_hand])

In [ ]:
data_path = os.path.join(os.getcwd(), 'data')

actions = np.array(['hello', 'thanks', 'namaste'])

num_sequence = 30
len_sequence = 30

In [ ]:
for action in actions:
    for sequence in range(num_sequence):
        try:
            os.makedirs(os.path.join(data_path, str(action), str(sequence)))
        except:
            pass

In [ ]:
capture = cv2.VideoCapture(0)
with mpipe_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
    
    for action in actions:
        for sequence in range(num_sequence):
            for frame_idx in range(len_sequence):
                
                # frame capture
                ret, frame = capture.read()

                result, img = mpipe_detection(frame, holistic)
                draw_landmarks(img, result)
                
                if frame_idx == 0:
                    cv2.putText(img, str(action), (120, 200),
                                cv2.FONT_HERSHEY_PLAIN, 4, (0, 255, 0), 4, cv2.LINE_AA)
                
                cv2.putText(img, 'Action: {} Sequence {}'.format(action, sequence), (15, 12),
                            cv2.FONT_HERSHEY_PLAIN, 0.5, (0, 0, 255), 1, cv2.LINE_AA)

                # exporting keypoints
                np_data = to_array_keypoints(result)
                np_path = os.path.join(data_path, str(action), str(sequence), str(frame_idx))
                np.save(np_path, np_data)

                cv2.imshow("Video", img)
                
                if frame_idx == 0:
                    cv2.waitKey(2000)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

    capture.release()
    cv2.destroyAllWindows()